# SK-6-ProcessFramework : Workflows et Orchestration

**Navigation** : [<< 05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | [Index](README.md) | [07-MultiModal >>](07-SemanticKernel-MultiModal.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre la difference entre **Plugins**, **Agents** et **Processes**
2. Creer des **Steps** (etapes) de workflow
3. Orchestrer des etapes avec **ProcessBuilder**
4. Gerer l'**etat** entre les etapes
5. Implementer des **conditions** et **branchements**

### Prerequis

- Python 3.10+
- Notebooks 01-05 completes
- Cle API OpenAI configuree (`.env`)

### Duree estimee : 40 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Plugins vs Agents vs Processes |
| 2 | Architecture | Steps, State, Events |
| 3 | ProcessBuilder | Creation de workflow |
| 4 | Gestion d'etat | Passage de donnees |
| 5 | Conditions | Branchements dynamiques |
| 6 | Cas d'usage | Validation de contenu |
| 7 | Conclusion | Resume, exercices |

> **Process Framework** : Introduit en preview dans SK 1.x, GA prevu Q2 2026. Ce framework permet d'orchestrer des workflows complexes avec gestion d'etat, checkpoints, et human-in-the-loop. Ideal pour les pipelines multi-etapes.

In [ ]:
# Installation
%pip install semantic-kernel python-dotenv --quiet

import os
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

# Chargement du fichier .env
load_dotenv("../.env")

kernel = Kernel()
kernel.add_service(OpenAIChatCompletion(service_id="default"))

# Settings par defaut pour les appels chat
default_settings = OpenAIChatPromptExecutionSettings()

print("Process Framework - Demo conceptuelle")

## 1. Introduction : Plugins vs Agents vs Processes

SK offre trois niveaux d'abstraction pour structurer votre code IA :

| Niveau | Abstraction | Granularite | Exemple |
|--------|-------------|-------------|---------|
| **Plugin** | Fonction | Atomique | `summarize_text()` |
| **Agent** | Entite autonome | Tache | "Resume ce document" |
| **Process** | Workflow | Pipeline | Generate -> Review -> Publish |

### Quand utiliser quoi ?

```
Complexite croissante --->

┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   PLUGIN    │    │   AGENT     │    │  PROCESS    │
│             │    │             │    │             │
│ - Fonction  │    │ - Autonome  │    │ - Pipeline  │
│ - Stateless │    │ - Stateful  │    │ - Multi-step│
│ - Simple    │    │ - Tools     │    │ - Checkpts  │
└─────────────┘    └─────────────┘    └─────────────┘
    "call()"         "invoke()"        "run()"
```

**Indicateurs pour utiliser un Process** :
- Pipeline avec plusieurs etapes sequentielles/paralleles
- Besoin de checkpoints (reprise apres echec)
- Human-in-the-loop (approbation humaine)
- Etat partage entre etapes
- Conditions de branchement complexes

## 2. Architecture du Process Framework

### Composants principaux

```
┌─────────────────────────────────────────────────────┐
│                    PROCESS                          │
│  ┌─────────────────────────────────────────────┐   │
│  │                 STATE                        │   │
│  │   (donnees partagees entre etapes)          │   │
│  └─────────────────────────────────────────────┘   │
│                                                     │
│  ┌─────────┐     ┌─────────┐     ┌─────────┐      │
│  │  STEP   │────>│  STEP   │────>│  STEP   │      │
│  │ Generate│     │ Review  │     │ Publish │      │
│  └─────────┘     └─────────┘     └─────────┘      │
│       ↓              ↓               ↓             │
│  ┌─────────────────────────────────────────────┐   │
│  │               EVENTS                         │   │
│  │  (declencheurs entre etapes)                │   │
│  └─────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────┘
```

| Composant | Role | Exemple |
|-----------|------|---------|
| **Process** | Conteneur du workflow | ContentValidationProcess |
| **Step** | Etape d'execution | GenerateStep, ReviewStep |
| **State** | Donnees partagees | `{"content": "...", "approved": False}` |
| **Event** | Declencheur | OnContentGenerated, OnReviewComplete |

## 3. Creation de Steps

Les Steps sont les briques de base des processes.

In [ ]:
from dataclasses import dataclass
from typing import Optional
from semantic_kernel.contents import ChatHistory

# Definition de l'etat du process
@dataclass
class ContentState:
    """Etat partage entre les etapes du workflow."""
    topic: str
    draft_content: Optional[str] = None
    review_feedback: Optional[str] = None
    is_approved: bool = False
    final_content: Optional[str] = None

# Step 1: Generation de contenu
async def generate_content_step(state: ContentState, kernel: Kernel) -> ContentState:
    """Genere un brouillon de contenu."""
    print(f"[Step 1] Generation du contenu sur: {state.topic}")
    
    chat_service = kernel.get_service(service_id="default")
    history = ChatHistory()
    history.add_user_message(f"Ecris un paragraphe court sur: {state.topic}")
    
    # SK 1.39+: settings requis
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=default_settings
    )
    state.draft_content = str(response[0])
    
    print(f"[Step 1] Brouillon genere: {len(state.draft_content)} caracteres")
    return state

# Step 2: Review du contenu
async def review_content_step(state: ContentState, kernel: Kernel) -> ContentState:
    """Evalue la qualite du contenu."""
    print(f"[Step 2] Review du contenu")
    
    chat_service = kernel.get_service(service_id="default")
    history = ChatHistory()
    history.add_user_message(
        f"""Evalue ce contenu et reponds par 'APPROVED' si c'est bon, sinon donne des suggestions:
        
        {state.draft_content}"""
    )
    
    # SK 1.39+: settings requis
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=default_settings
    )
    state.review_feedback = str(response[0])
    state.is_approved = "approved" in state.review_feedback.lower()
    
    print(f"[Step 2] Approuve: {state.is_approved}")
    return state

# Step 3: Publication
async def publish_content_step(state: ContentState) -> ContentState:
    """Publie le contenu final."""
    print(f"[Step 3] Publication du contenu")
    
    if state.is_approved:
        state.final_content = state.draft_content
        print(f"[Step 3] Contenu publie avec succes!")
    else:
        print(f"[Step 3] Contenu rejete - necessite revision")
    
    return state

print("Steps definis: generate, review, publish")

In [ ]:
# Execution manuelle du pipeline
async def run_content_pipeline(topic: str, kernel: Kernel) -> ContentState:
    """Execute le pipeline de validation de contenu."""
    
    # Initialiser l'etat
    state = ContentState(topic=topic)
    
    # Etape 1: Generation
    state = await generate_content_step(state, kernel)
    
    # Etape 2: Review
    state = await review_content_step(state, kernel)
    
    # Etape 3: Publication (conditionnelle)
    state = await publish_content_step(state)
    
    return state

# Test du pipeline
result = await run_content_pipeline("Les avantages de Semantic Kernel", kernel)

print("\n" + "=" * 60)
print("RESULTAT DU PIPELINE")
print("=" * 60)
print(f"Topic: {result.topic}")
print(f"Approuve: {result.is_approved}")
print(f"Contenu final: {'Oui' if result.final_content else 'Non'}")

### Interpretation : Pipeline de base

Le pipeline ci-dessus illustre les concepts fondamentaux :

| Etape | Action | Modifie l'etat |
|-------|--------|----------------|
| **Generate** | Cree le brouillon | `draft_content` |
| **Review** | Evalue la qualite | `review_feedback`, `is_approved` |
| **Publish** | Publie si approuve | `final_content` |

**Points cles** :
- L'etat (`ContentState`) est passe entre les etapes
- Chaque etape peut lire ET modifier l'etat
- Les conditions sont evaluees sur l'etat (`is_approved`)

## 4. Process avec iterations

Un pattern courant : iterer jusqu'a ce qu'une condition soit remplie.

In [ ]:
async def run_iterative_pipeline(
    topic: str, 
    kernel: Kernel, 
    max_iterations: int = 3
) -> ContentState:
    """Pipeline avec boucle de revision."""
    
    state = ContentState(topic=topic)
    
    for iteration in range(max_iterations):
        print(f"\n--- Iteration {iteration + 1}/{max_iterations} ---")
        
        # Generation (ou revision)
        if iteration == 0:
            state = await generate_content_step(state, kernel)
        else:
            # Reviser en tenant compte du feedback
            state = await revise_content_step(state, kernel)
        
        # Review
        state = await review_content_step(state, kernel)
        
        # Condition de sortie
        if state.is_approved:
            print(f"\n[SUCCESS] Contenu approuve apres {iteration + 1} iteration(s)")
            break
    
    # Publication
    state = await publish_content_step(state)
    
    return state

async def revise_content_step(state: ContentState, kernel: Kernel) -> ContentState:
    """Revise le contenu selon le feedback."""
    print(f"[Step R] Revision du contenu")
    
    chat_service = kernel.get_service(service_id="default")
    history = ChatHistory()
    history.add_user_message(
        f"""Revise ce contenu en tenant compte du feedback:
        
        CONTENU ORIGINAL:
        {state.draft_content}
        
        FEEDBACK:
        {state.review_feedback}
        
        Produis une version amelioree."""
    )
    
    # SK 1.39+: settings requis
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=default_settings
    )
    state.draft_content = str(response[0])
    
    print(f"[Step R] Contenu revise")
    return state

# Test du pipeline iteratif
result = await run_iterative_pipeline("L'importance du RAG en IA", kernel, max_iterations=3)

print("\n" + "=" * 60)
print(f"Resultat final: {'PUBLIE' if result.final_content else 'NON PUBLIE'}")

### Interpretation : Pipeline iteratif

Ce pattern "Generate-Review-Revise" est courant :

```
┌──────────┐     ┌──────────┐
│ Generate │────>│  Review  │
└──────────┘     └────┬─────┘
      ^               │
      │    ┌──────────┴──────────┐
      │    │                     │
      │    v                     v
  ┌───────────┐           ┌───────────┐
  │  Revise   │           │  Publish  │
  │(not OK)   │           │   (OK)    │
  └───────────┘           └───────────┘
```

**Applications** :
- Code review automatique (Coder ↔ Reviewer)
- Redaction de contenu (Writer ↔ Editor)
- Validation de donnees (Extractor ↔ Validator)

## 5. Pattern avec Human-in-the-Loop (conceptuel)

Le Process Framework permet d'integrer des points d'approbation humaine.

In [ ]:
# Simulation d'approbation humaine
async def human_approval_step(state: ContentState) -> ContentState:
    """Demande une approbation humaine (simule)."""
    print(f"\n[HUMAN] Contenu a approuver:")
    print("-" * 40)
    print(state.draft_content[:200] + "..." if len(state.draft_content) > 200 else state.draft_content)
    print("-" * 40)
    
    # En production, ceci serait un webhook, email, ou UI
    # Ici on simule une approbation automatique
    print("[HUMAN] Approbation simulee...")
    state.is_approved = True
    
    return state

# Pipeline avec approbation humaine
async def run_human_in_loop_pipeline(topic: str, kernel: Kernel) -> ContentState:
    """Pipeline avec point d'approbation humaine."""
    
    state = ContentState(topic=topic)
    
    # Generation
    state = await generate_content_step(state, kernel)
    
    # Review automatique
    state = await review_content_step(state, kernel)
    
    # Point d'arret pour approbation humaine
    if not state.is_approved:
        print("\n[PROCESS] Review automatique negative - demande approbation humaine")
        state = await human_approval_step(state)
    
    # Publication
    state = await publish_content_step(state)
    
    return state

# Test
result = await run_human_in_loop_pipeline("Les limites des LLMs", kernel)
print(f"\nContenu final publie: {'Oui' if result.final_content else 'Non'}")

## 6. Comparaison avec alternatives

| Framework | Forces | Cas d'usage |
|-----------|--------|-------------|
| **SK Process** | Integration SK native, Dapr support | Workflows IA simples |
| **LangGraph** | Graphes complexes, cycles | Multi-agent sophistique |
| **Prefect/Airflow** | Scheduling, monitoring | Data pipelines |
| **Temporal** | Durabilite, versioning | Workflows long-running |
| **AutoGen Flows** | Multi-agent natif | Conversations complexes |

### Quand utiliser SK Process Framework ?

| OUI | NON |
|-----|-----|
| Deja dans l'ecosysteme SK | Besoins de scheduling avance |
| Pipelines lineaires simples | Graphes avec cycles complexes |
| Human-in-the-loop basique | Monitoring entreprise |
| Prototypage rapide | Production critique |

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **Process** | Workflow orchestre | Conteneur de steps |
| **Step** | Etape d'execution | Fonction async |
| **State** | Donnees partagees | Dataclass mutable |
| **Condition** | Branchement | `if state.is_approved:` |
| **Iteration** | Boucle jusqu'a succes | `for i in range(max):` |

## Points cles a retenir

1. **Plugins < Agents < Processes** - Choisir le bon niveau d'abstraction
2. **L'etat lie les etapes** - Dataclass partagee entre steps
3. **Les conditions permettent le branchement** - Logique metier dans l'etat
4. **Human-in-the-loop est essentiel** - Points d'arret pour approbation
5. **Preview API** - Peut evoluer avant GA (Q2 2026)

## Exercices suggeres

1. **Pipeline de traduction** : Translate -> Review -> Proofread -> Publish
2. **Multi-agent process** : Integrer AgentGroupChat dans un step
3. **Persistance** : Sauvegarder l'etat entre les executions

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [07-MultiModal](07-SemanticKernel-MultiModal.ipynb) | DALL-E, Whisper, Vision |
| [08-MCP](08-SemanticKernel-MCP.ipynb) | Model Context Protocol |

---

**Navigation** : [<< 05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | [Index](README.md) | [07-MultiModal >>](07-SemanticKernel-MultiModal.ipynb)